In [ ]:
from diags import Conventional
import numpy as np
import pandas as pd
from filter_df import filter_df
from make_da_plots import make_base_plots, make_wind_base_plots
import importlib
from plot_driver import da_base_plots
import time_series as time_series
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import xarray as xr

In [ ]:
pd.set_option('display.max_columns', None) # so I can see all columns on the df

# Interactive Plot Making

### Get date

Use current date and time (most recent hour) or put in your own custom time, note that these directories only have diagnostic files from today and the previous 2 days

In [ ]:
variable = 't'
# Current time (most recent hour)
# now_utc = datetime.now(timezone.utc)
# ex_time = now_utc.replace(hour=0, minute=0, second=0, microsecond=0)
# date_str = ex_time.strftime('%Y%m%d%H')

# Custom time
date_str = '2024080814' 

In [ ]:
day_str = date_str[:8]
hour_str = date_str[-2:]

### RTMA GSI diag file paths

In [ ]:
#diag files on jet
rtma_diag_ges_fp = f'/lfs4/BMC/nrtrr/NCO_dirs/rtma.v0.8.8/com/prod/RTMA_CONUS.{day_str}/{hour_str}/diag_conv_{variable}_ges.{date_str}.nc4.gz'
rtma_diag_anl_fp = f'/lfs4/BMC/nrtrr/NCO_dirs/rtma.v0.8.8/com/prod/RTMA_CONUS.{day_str}/{hour_str}/diag_conv_{variable}_anl.{date_str}.nc4.gz'

### Query data
Print the total number of obs before filtering

In [ ]:
rtma_diag_ges = Conventional(rtma_diag_ges_fp)
rtma_diag_anl = Conventional(rtma_diag_anl_fp)
rtma_df_ges = rtma_diag_ges.get_data()
rtma_df_anl = rtma_diag_anl.get_data()

In [ ]:
print(len(rtma_df_ges))

In [ ]:
print(len(rtma_df_anl))

## Filtering dataframes

### Set filter arguments
Here are various example filters that can be passed to filter_df, feel free to try adding or removing any

In [ ]:
station_ids = ['NLOYR3BA']
obs_types_sur_temp = [181, 187, 188, 195]
obs_types_air_temp = [126, 133]
use = 1
elv_range = (0,20)
p_range = (500,1000)
err_range = (0, 4)

# Lat and lon ranges for colorado
co_lats = (37, 41)
co_lons = (251, 258) #to convert from deg west to deg east (360 - deg west)

### Filter df
Filter the df and see how many obs you end up with, sometimes there are slight difference in number of obs between anl and ges dfs

In [ ]:
# temp examples
# rtma_fil_dfs = filter_df([rtma_df_anl, rtma_df_ges], hem='CONUS', obs_types=obs_types_air_temp, use=1)
rtma_fil_dfs = filter_df([rtma_df_anl, rtma_df_ges], obs_types=obs_types_sur_temp, use=1,
                    lat_range=co_lats,lon_range=co_lons)
    
rtma_df_anl_fil = rtma_fil_dfs[0]
rtma_df_ges_fil = rtma_fil_dfs[1]
print(len(rtma_df_anl_fil))
print(len(rtma_df_ges_fil))

## Making base plots
Now run make_base_plots to create your plots, uncomment the top cell to save the plots as files. The returned object shared_norm is the normalization used for the scales of oma/omf maps, you can pass this to another call to make_base_plots if you want the same scale to be use for comparison purposes

In [ ]:
# make_base_plots([rtma_df_ges_fil, rtma_df_anl_fil], diag_ges.metadata, zoom=False, save_plots=True)

In [ ]:
to_share_norm = make_base_plots([rtma_df_ges_fil, rtma_df_anl_fil], rtma_diag_ges.metadata, zoom=False)

# RRFS Diag Files
Now lets do the same process but with RRFS diag files

In [ ]:
#temperature diag files jet
rrfs_diag_ges_fp = f'/lfs4/BMC/nrtrr/NCO_dirs/v0.9.5/com/prod/rrfs.{day_str}/{hour_str}/diag_conv_{variable}_ges.{date_str}.nc4.gz'
rrfs_diag_anl_fp = f'/lfs4/BMC/nrtrr/NCO_dirs/v0.9.5/com/prod/rrfs.{day_str}/{hour_str}/diag_conv_{variable}_anl.{date_str}.nc4.gz'

## Query data
Print the total number of obs before filtering

In [ ]:
rrfs_diag_ges = Conventional(rrfs_diag_ges_fp)
rrfs_diag_anl = Conventional(rrfs_diag_anl_fp)
rrfs_df_ges = rrfs_diag_ges.get_data()
rrfs_df_anl = rrfs_diag_anl.get_data()

In [ ]:
print(len(rrfs_df_ges))

In [ ]:
print(len(rrfs_df_anl))

## Filtering dataframes

### Filter df

In [ ]:
# temp examples
# rrfs_fil_dfs = filter_df([rrfs_df_anl, rrfs_df_ges], use =1)
rrfs_fil_dfs = filter_df([rrfs_df_anl, rrfs_df_ges], obs_types=obs_types_sur_temp, use=1,
                    lat_range=co_lats,lon_range=co_lons)

rrfs_df_anl_fil = rrfs_fil_dfs[0]
rrfs_df_ges_fil = rrfs_fil_dfs[1]
print(len(rrfs_df_anl_fil))
print(len(rrfs_df_ges_fil))

### Make plots
Here we pass the to_share_norm that was returned by the make_base_plots we called for RTMA, this will cause the plots created for RRFS to have the same scale for the oma and omf plots

In [ ]:
# make_base_plots([rrfs_df_ges_fil, rrfs_df_anl_fil], diag_ges.metadata, shared_norm = to_share_norm, save_plots=True, zoom=True)

In [ ]:
make_base_plots([rrfs_df_ges_fil, rrfs_df_anl_fil], rrfs_diag_ges.metadata, shared_norm = to_share_norm, zoom=False)

## Wind plots
Wind plots require a slightly different process due to the different format with u and v components, here is an example making base plots for RTMA wind diagnostic data

In [ ]:
#wind diag files on jet
diag_uv_ges_fp = f'/lfs4/BMC/nrtrr/NCO_dirs/rtma.v0.8.8/com/prod/RTMA_CONUS.{day_str}/{hour_str}/diag_conv_uv_ges.{date_str}.nc4.gz'
diag_uv_anl_fp = f'/lfs4/BMC/nrtrr/NCO_dirs/rtma.v0.8.8/com/prod/RTMA_CONUS.{day_str}/{hour_str}/diag_conv_uv_anl.{date_str}.nc4.gz'

In [ ]:
#Wind Diag file
diag_uv_ges = Conventional(diag_uv_ges_fp)
diag_uv_anl = Conventional(diag_uv_anl_fp)
df_uv_ges = diag_uv_ges.get_data()
df_uv_anl = diag_uv_anl.get_data()

In [ ]:
df_uv_anl

In [ ]:
# wind examples
# fil_uv_dfs = filter_df([df_uv_anl, df_uv_ges], obs_types= [287], hem='CONUS', use=1)
fil_uv_dfs = filter_df([df_uv_anl, df_uv_ges], obs_types= [287], lat_range=co_lats, lon_range=co_lons)

df_uv_anl_fil = fil_uv_dfs[0]
df_uv_ges_fil = fil_uv_dfs[1]
print(len(df_uv_anl_fil))
print(len(df_uv_ges_fil))

In [ ]:
# make_wind_base_plots([df_uv_anl_fil, df_uv_ges_fil], diag_uv_ges.metadata, save_plots=True)

In [ ]:
make_wind_base_plots([df_uv_ges_fil, df_uv_anl_fil], diag_uv_ges.metadata)

# JEDI hofx files
This same workflow can be done for JEDI hofx file with some more small changes, for example the JEDI diagnostics are all in one file instead of two like GSI. So you only have to read, filter, and pass one df. There are also some small syntactic differences, please see the notebook titled jedi_diag_explore for an example reading, filtering and making plots using JEDI files